In [1]:
import numpy as np
import pandas
from gurobipy import *

## Exemple jouet20

In [45]:
P = np.array(( ((0,0,1),(0,1,0 ),(0,0,1)) , ((0,1,0),(0,1,0 ),(0,0,1)) , ((1,0,0),(1,0,0 ),(0,0,1)), ((1,0,0),(0,1,0),(0.3333,0.3333,0.3334))  ))
R = np.array(((2, 1, -1000, -1000), (-1000, -1000,2,-1000),(-1000,-1000,-1000,3)))
gamma = 1
print("matrices de transitions:")
print(P)
print("matrices de rewards:")
print(R)

matrices de transitions:
[[[0.     0.     1.    ]
  [0.     1.     0.    ]
  [0.     0.     1.    ]]

 [[0.     1.     0.    ]
  [0.     1.     0.    ]
  [0.     0.     1.    ]]

 [[1.     0.     0.    ]
  [1.     0.     0.    ]
  [0.     0.     1.    ]]

 [[1.     0.     0.    ]
  [0.     1.     0.    ]
  [0.3333 0.3333 0.3334]]]
matrices de rewards:
[[    2     1 -1000 -1000]
 [-1000 -1000     2 -1000]
 [-1000 -1000 -1000     3]]


In [43]:
'''
si n est la taille de l'espace d'état il y a n+1 variables de décision
le scalaire g et h(s) avec s les n etats.
g terme de la valeure moyenne
pour résoudre le systeme:
min g
sous contrainte
h(s) >= r(s,a) -g + \sum_{s'} p(s'|s,a) h(s') pour tout s et a
'''

nbS = 3
nbA = 4
m = Model("MDPjouet20")     

# declaration variables de decision
v = []
for i in range(nbS):
    v.append(m.addVar(vtype=GRB.CONTINUOUS)) 

g = []
g.append(m.addVar(vtype=GRB.CONTINUOUS))

# maj du modele pour integrer les nouvelles variables
m.update()

obj = LinExpr();

obj = g[0]

# definition de l'objectif
m.setObjective(obj,GRB.MINIMIZE)

# Definition des contraintes
for i in range(nbS):
    for j in range(nbA):
        sums = 0
        for k in range(nbS):
            sums = sums + gamma*P[j][i,k]*v[k]
        m.addConstr( v[i] >= R[i,j] -g[0] + sums, "Contrainte%d" % i) #r(s,a) -g + \sum_{s'} p(s'|s,a) h(s')

# Resolution
m.optimize()

print("")                
print('Solution optimale:')
for i in range(nbS):
        print('v%d'%(i+1), '=', v[i])
#print('Valeur de la fonction objectif (somme des V(S)) :', m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12 rows, 4 columns and 21 nonzeros
Model fingerprint: 0x12c6caf3
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 8 rows and 0 columns
Presolve time: 0.03s
Presolved: 4 rows, 4 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.333200e+00   0.000000e+00      0s
       2    2.5000250e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.05 seconds
Optimal objective  2.500025001e+00

Solution optimale:
v1 = <gurobi.Var C0 (value 0.5000250012500622)>
v2 = <gurobi.Var C1 (value 0.0)>
v3 = <gurobi.Var C2 (value 1.0000500025001244)>
